In [191]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import re
import os
import csv

from tqdm.notebook import tqdm

In [192]:
DATA_PATH = 'outputs/vdjserver.csv'

data = pd.read_csv(DATA_PATH, lineterminator="\n")

print(f'Loaded dataset with {len(data)} rows')

Loaded dataset with 65 rows


In [193]:
with open('EDAM/edam_topics.txt', 'r') as f:
    edam_topics = [topic.strip() for topic in f.readlines()]

quoted_topics = [topic for topic in edam_topics if topic.startswith('"') and topic.endswith('"')]

# Remove quotes
edam_topics = [topic[1:-1] if topic.startswith('"') and topic.endswith('"') else topic for topic in edam_topics]

## Format Outputs

Split outputs on tab, and check for other separators that GPT may have used in error.

In [194]:
data['Predictions'] = data['Predictions'].str.replace('\\t', '\t')


In [195]:
def split_topics(topics):
    cleaned_topics = [topic.strip() for topic in topics.split('\t')]
    for i in range(len(cleaned_topics)):
        for quoted_topic in quoted_topics:
            if quoted_topic.replace('\"', '').lower() in cleaned_topics[i].lower():
                cleaned_topics[i] = cleaned_topics[i].replace(quoted_topic.replace('\"', ''), quoted_topic)
                break
            else:
                cleaned_topics[i] = cleaned_topics[i].replace('\"', '')
    return cleaned_topics

data['Predictions'] = data['Predictions'].apply(split_topics)

In [196]:
separators = ['    ', '   ', '  ', '\n', '<TAB>', 'TAB', '<tab>', '(tab)', '<Tab>', '[tab]', '▪️', '(Tab)', '\xa0\xa0\xa0\xa0', '\xa0', '\u2003', '、', '\x0b', '\x0c', ';', '.', '--', '-', '–', '_', '\\', '\\n', '/', '@', '|', '\r', '+', '<', '>', '·']

# Join the separators with the regex OR operator |
sep_pattern = '|'.join(map(re.escape, separators))

for i in tqdm(range(len(data))):
    # Replace commas not enclosed in double quotes with |
    data['Predictions'][i] = [re.sub(r',(?=(?:[^"]*"[^"]*")*[^"]*$)', '|', pred) for pred in data['Predictions'][i]]
    # Split on the separators
    split_list = [re.split(sep_pattern, pred) for pred in data['Predictions'][i]]
    # Flatten the list
    data['Predictions'][i] = [item for sublist in split_list for item in sublist]

data['Predictions'] = data['Predictions'].apply(lambda x: [item.strip() for item in x])

data['Predictions'] = data['Predictions'].apply(lambda x: [re.sub(r'Category \d+:', '', pred) for pred in x])

  0%|          | 0/65 [00:00<?, ?it/s]

In [197]:
is_one_dimensional = all(isinstance(pred, str) for preds in data['Predictions'] for pred in preds)
print(is_one_dimensional)

True


In [198]:
# ## Capture any weirdly formatted outputs (using the wrong separators)

# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('    ') if 0 < len(x) <= 1 and '    ' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('   ') if 0 < len(x) <= 1 and '   ' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('  ') if 0 < len(x) <= 1 and '  ' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\n') if 0 < len(x) <= 1 and '\n' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('<TAB>') if 0 < len(x) <= 1 and '<TAB>' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('TAB') if 0 < len(x) <= 1 and 'TAB' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('<tab>') if 0 < len(x) <= 1 and '<tab>' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('(tab)') if 0 < len(x) <= 1 and '(tab)' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('<Tab>') if 0 < len(x) <= 1 and '<Tab>' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('[tab]') if 0 < len(x) <= 1 and '[tab]' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('▪️') if 0 < len(x) <= 1 and '▪️' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('(Tab)') if 0 < len(x) <= 1 and '<Tab>' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\xa0\xa0\xa0\xa0') if 0 < len(x) <= 1 and '\xa0\xa0\xa0\xa0' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\xa0') if 0 < len(x) <= 1 and '\xa0' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\u2003') if 0 < len(x) <= 1 and '\u2003' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('、') if 0 < len(x) <= 1 and '、' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\x0b') if 0 < len(x) <= 1 and '\x0b' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\x0c') if 0 < len(x) <= 1 and '\x0c' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split(';') if 0 < len(x) <= 1 and ';' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('.') if 0 < len(x) <= 1 and '.' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('--') if 0 < len(x) <= 1 and '--' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('-') if 0 < len(x) <= 1 and '-' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('–') if 0 < len(x) <= 1 and '–' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('_') if 0 < len(x) <= 1 and '_' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\\') if 0 < len(x) <= 1 and '\\' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\\n') if 0 < len(x) <= 1 and '\\n' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('/') if 0 < len(x) <= 1 and '/' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('@') if 0 < len(x) <= 1 and '@' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('|') if 0 < len(x) <= 1 and '|' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('\r') if 0 < len(x) <= 1 and '\r' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('+') if 0 < len(x) <= 1 and '+' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('<') if 0 < len(x) <= 1 and '<' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split('>') if 0 < len(x) <= 1 and '>' in list(x)[0] else x)
# data['Predictions'] = data['Predictions'].apply(lambda x: [pred.strip() for pred in csv.reader([list(x)[0]],skipinitialspace=True, delimiter=',', quotechar='"').__next__()])
# data['Predictions'] = data['Predictions'].apply(lambda x: [re.sub(r'Category \d+:', '', pred) for pred in x])
# # data['Predictions'] = data['Predictions'].apply(lambda x: list(x)[0].split(', ') if len(x) <= 1 and ', ' in list(x)[0] else x)

In [199]:
# check if any of the quoted topics, or their equivalent without quotes, 
# are in any of the prediction sets with length less than or equal to 1. ""
# If there is, then add the quotes back in if they don't have them,
#  and then split on commas while avoiding anything inside quotes
def process_predictions(predictions):
    processed_predictions = []
    for prediction in predictions:
        formatted = False
        for topic in quoted_topics:
            formatted_topic = topic.replace('\"', '')
            if formatted_topic in prediction:
                processed_prediction = prediction.replace(formatted_topic, f'{topic}')
                processed_predictions.append(processed_prediction)
                formatted = True
                break
        if not formatted:
            processed_predictions.append(prediction)
        
    final_predictions = []
    for prediction in processed_predictions:
        if '\"' in prediction:
            parts = re.findall(r'[^"]+|"[^"]+"', prediction)
            final_predictions.extend(parts)
        else:
            final_predictions.extend([pred.strip() for pred in prediction.split(',')])
    return set(final_predictions)

data['Predictions'] = data['Predictions'].apply(process_predictions)

In [200]:
filtered_predictions = data[data['Predictions'].apply(len) <= 1]['Predictions']

# Filter out any expected predictions. May or may not be hallucinations
unexpected_predictions = []
for original_index, pred_set in filtered_predictions.items():
    lst = list(pred_set)
    if len(lst) == 0:
        continue
    prediction = lst[0]
    if '\"' not in prediction and ' ' in prediction and prediction not in edam_topics:
        unexpected_predictions.append((original_index, prediction))

# Print the unexpected predictions and their corresponding original indices
count = len(unexpected_predictions)
print(f"Number of unexpected predictions: {count}")
for original_index, prediction in unexpected_predictions:
    print(f"Original Index: {original_index}, Prediction: {prediction}")

Number of unexpected predictions: 0


## Hallucinations

Filter out topics not in the EDAM topics list. The filtered topics may be matched to a topic or synonym->topic in the next section.

In [201]:
data['Hallucinations'] = data['Predictions'].apply(lambda preds: set([pred.replace('.', '').replace('\"', '') for pred in preds if pred.replace('.', '').replace('\"', '') not in edam_topics]))

In [202]:
data['Predictions'] = data['Predictions'].apply(lambda preds: set([pred.replace('.', '').replace('\"', '') for pred in preds if pred.replace('.', '').replace('\"', '') in edam_topics]))
data['Predictions'] = data.apply(lambda row: set([topic for topic in row['Predictions'] if topic not in row['Hallucinations']]), axis=1)

## Synonym matching

Check for mispelled/misformatted topics or synonyms using levenshtein

In [203]:
edam = pd.read_csv('EDAM/EDAM.csv')

edam = edam[edam['Class ID'].str.contains('topic')].reset_index(drop=True)
# edam['Preferred Label'].apply(lambda topic: topic.replace('\"', ''))
edam = edam[edam['Preferred Label'].isin([topic.replace('\"', '') for topic in edam_topics])].reset_index(drop=True)

In [204]:
edam['Synonyms'] = edam['Synonyms'].fillna('').apply(lambda x: x.split('|') if x != '' else [])

In [205]:
missing_topics = set([topic.replace('\"', '') for topic in edam_topics]) - set(edam['Preferred Label'])
missing_topics

set()

In [206]:
synonym_dict = {}

for index, row in edam.iterrows():
    for synonym in row['Synonyms']:
        synonym_dict[synonym] = row['Preferred Label']

In [207]:
synonym_dict

{'Molecular docking': 'Molecular modelling',
 'Homology modeling': 'Molecular modelling',
 'Docking': 'Molecular modelling',
 'Comparative modelling': 'Molecular modelling',
 'Homology modelling': 'Molecular modelling',
 'Evolution': 'Evolutionary biology',
 'Freshwater science': 'Freshwater biology',
 'Nutrition': 'Nutritional science',
 'Nutrition science': 'Nutritional science',
 'Dietetics': 'Nutritional science',
 'Cardiovascular medicine': 'Cardiology',
 'Heart disease': 'Cardiology',
 'Cardiovascular disease': 'Cardiology',
 'Gene features': 'Gene structure',
 'Fusion genes': 'Gene structure',
 'Transcriptome': 'Transcriptomics',
 'Comparative transcriptomics': 'Transcriptomics',
 'Ancestral genomes': 'Paleogenomics',
 'Paleogenetics': 'Paleogenomics',
 'Panomics': 'Multiomics',
 'Pan-omics': 'Multiomics',
 'Integrative omics': 'Multiomics',
 'Multi-omics': 'Multiomics',
 'RNA metabarcoding': 'Metabarcoding',
 'DNA metabarcoding': 'Metabarcoding',
 'eDNA metabarcoding': 'Metabar

In [208]:
import Levenshtein

hallucinations = data['Hallucinations']

matched_topics = {}

for hallucination_set in tqdm(hallucinations):
    for hallucination in hallucination_set:
        if hallucination in matched_topics:
            continue
        matched = False
        # First check for a match in the topics list
        sorted_topics = sorted(edam_topics, key=lambda topic: Levenshtein.distance(hallucination, topic))
        for topic in sorted_topics:
            distance = Levenshtein.distance(hallucination, topic)
            if  0 < distance <= 2:
                matched_topics[hallucination] = topic
                matched = True
                break
        
        # If the hallucination has already been matched, skip to the next hallucination
        if matched:
            continue
        
        # If no match in the topics list, look through the available synonyms
        sorted_synonyms = sorted(synonym_dict.keys(), key=lambda topic: Levenshtein.distance(hallucination, topic))
        for topic in sorted_synonyms:
            distance = Levenshtein.distance(hallucination, topic)
            if 0 <= distance <= 1:
                matched_topics[hallucination] = synonym_dict[topic]
                matched = True
                break     

        if matched:
            continue

        for topic in sorted_topics:
            if topic.lower() in hallucination.lower().split():
                matched_topics[hallucination] = topic
                break
        # No break reached
        else:
            for topic in sorted_synonyms:
                if topic.lower() in hallucination.lower().split():
                    matched_topics[hallucination] = synonym_dict[topic]
                    break

matched_topics

  0%|          | 0/65 [00:00<?, ?it/s]

{'Disease': 'Pathology',
 'Iinfectious disease': 'Infectious disease',
 'Infectious Disease': 'Infectious disease',
 'T cell biology': 'Biology',
 'Cancer Study': 'Oncology',
 'Cancer': 'Oncology',
 'Clinical Allergy': 'Allergy, clinical immunology and immunotherapeutics',
 'Lung cancer': 'Oncology',
 'Clinical medicine': 'Medicine'}

In [209]:
for index, row in data.iterrows():
    if len(row['Hallucinations']) > 0:
        for hallucination in list(row['Hallucinations']):
            if hallucination in matched_topics:
                print(f"'{hallucination}' in row {index} matches topic '{matched_topics[hallucination]}'")
                data.at[index, 'Predictions'].add(matched_topics[hallucination])
                continue

'Disease' in row 7 matches topic 'Pathology'
'Iinfectious disease' in row 8 matches topic 'Infectious disease'
'Infectious Disease' in row 20 matches topic 'Infectious disease'
'T cell biology' in row 24 matches topic 'Biology'
'Cancer Study' in row 35 matches topic 'Oncology'
'Cancer Study' in row 36 matches topic 'Oncology'
'Cancer' in row 37 matches topic 'Oncology'
'Cancer Study' in row 41 matches topic 'Oncology'
'Clinical Allergy' in row 43 matches topic 'Allergy, clinical immunology and immunotherapeutics'
'Cancer Study' in row 46 matches topic 'Oncology'
'Lung cancer' in row 46 matches topic 'Oncology'
'Clinical medicine' in row 53 matches topic 'Medicine'
'Cancer' in row 58 matches topic 'Oncology'


In [210]:
data[data['Hallucinations'].apply(len) > 1]

,_id,Name,Description,Model,Predictions,Hallucinations
35,VDJ_PRJNA315543,"Sims et al., Diversity and divergence of the glioma-infiltrating T-cell receptor repertoire","Cancer Study\n nonneoplastic, low-grade and high-grade glioma\n NIH/National Institute of Biomedical Imaging and Bioengineering Grant K01EB016071, NIH/National Institute of Neurological Disorders and Stroke Grant R01NS066955",gpt-3.5-turbo,"{Genomics, Oncology}","{Immuno…strain, Cancer Study}"
46,VDJ_PRJNA506151,"Jia et al., Local mutational diversity drives intratumoral immune heterogeneity in non-small cell lung cancer.","Cancer Study\n Lung cancer\n National Nature Science Foundation of China (81222031) to B.Z., and Beau Biden Cancer Moonshot Initiative from the NIH (R33CA225328)",gpt-3.5-turbo,"{Immunology, Oncology}","{Cancer Study, Lung cancer}"


In [211]:
# Add quotes back in for the predictions with commas (i.e, "Data submission, annotation, and curation")
data['Predictions'] = data['Predictions'].apply(lambda preds: [f'"{pred}"' if f'"{pred}"' in quoted_topics else pred for pred in preds])

In [212]:
data[data['Hallucinations'].apply(len) > 0][['Predictions', 'Hallucinations']]

,Predictions,Hallucinations
7,"[Drug discovery, Bioengineering, Pathology]",{Disease}
8,"[Immunology, Infectious disease, Transcriptomics]",{Iinfectious disease}
20,"[Vaccinology, Immunology, Infectious disease]",{Infectious Disease}
24,"[Biology, Immunology, Biomedical science]",{T cell biology}
25,"[Immunology, Neurology]",{Multiple Sclerosis Study}
32,"[Immunogenetics, Genetics]",{Immune system}
35,"[Genomics, Oncology]","{Immuno…strain, Cancer Study}"
36,"[Immunology, Oncology, Data acquisition]",{Cancer Study}
37,"[Immunology, Oncology, Bioinformatics]",{Cancer}
41,"[Multiomics, Oncology]",{Cancer Study}


In [213]:
data['Predictions'] = data['Predictions'].apply(lambda lst: set(lst))

In [214]:
file_name = os.path.basename(DATA_PATH).replace('.', '_processed.')

data.to_csv(f'outputs/{file_name}', index=False)